# DuckQL Notebook Usage

This notebook demonstrates how to use DuckQL for data analysis with DuckDB and GraphQL.

In [ ]:
# Setup
import duckdb
import pandas as pd
import asyncio
import nest_asyncio
from duckql import DuckQL

# Enable async in notebooks
nest_asyncio.apply()

## 1. Create Sample Database

In [ ]:
# Create an in-memory database with sample data
conn = duckdb.connect(":memory:")

# Sales data
conn.execute("""
    CREATE TABLE sales AS 
    SELECT 
        DATE '2024-01-01' + INTERVAL (i % 30) DAY as date,
        CASE (i % 3) 
            WHEN 0 THEN 'Widget'
            WHEN 1 THEN 'Gadget' 
            ELSE 'Gizmo'
        END as product,
        CASE (i % 5)
            WHEN 0 THEN 'North'
            WHEN 1 THEN 'South'
            WHEN 2 THEN 'East'
            WHEN 3 THEN 'West'
            ELSE 'Central'
        END as region,
        ROUND(100 + RANDOM() * 900, 2) as amount,
        ROUND(1 + RANDOM() * 50) as quantity
    FROM generate_series(1, 1000) as t(i)
""")

# Customer data with JSON
conn.execute("""
    CREATE TABLE customers (
        id INTEGER PRIMARY KEY,
        name VARCHAR,
        email VARCHAR,
        metadata JSON,
        tags VARCHAR[],
        created_at TIMESTAMP
    )
""")

conn.execute("""
    INSERT INTO customers VALUES
    (1, 'Alice Johnson', 'alice@example.com', 
     '{"tier": "gold", "preferences": {"notifications": true}}',
     ['vip', 'early-adopter'], '2024-01-15 10:00:00'),
    (2, 'Bob Smith', 'bob@example.com',
     '{"tier": "silver", "preferences": {"notifications": false}}',
     ['regular'], '2024-01-20 14:30:00'),
    (3, 'Carol White', 'carol@example.com',
     '{"tier": "platinum", "preferences": {"notifications": true, "newsletter": true}}',
     ['vip', 'beta-tester', 'frequent-buyer'], '2024-01-10 09:15:00')
""")

print("Database created with sample data!")

## 2. Initialize DuckQL

In [ ]:
# Create DuckQL instance
duckql = DuckQL(conn)

# Helper function for running queries in notebook
async def query(graphql_query: str, variables=None):
    """Execute a GraphQL query and return the data."""
    schema = duckql.get_schema()
    result = await schema.execute(graphql_query, variable_values=variables)
    
    if result.errors:
        print("Errors:", result.errors)
        return None
    
    return result.data

# Sync wrapper for easier notebook use
def run_query(graphql_query: str, variables=None):
    """Synchronous query execution for notebooks."""
    return asyncio.run(query(graphql_query, variables))

## 3. Explore Schema with GraphQL

In [ ]:
# Get all sales data
data = run_query("""
    query {
        sales(limit: 5, orderBy: { date: DESC }) {
            date
            product
            region
            amount
            quantity
        }
    }
""")

# Convert to DataFrame
df = pd.DataFrame(data['sales'])
df

## 4. Filter and Analyze Data

In [ ]:
# Query with filters
data = run_query("""
    query {
        sales(
            where: {
                _and: [
                    { product: { eq: "Widget" } }
                    { amount: { gte: 500 } }
                    { region: { in: ["North", "South"] } }
                ]
            }
            orderBy: { amount: DESC }
            limit: 10
        ) {
            date
            region
            amount
            quantity
        }
    }
""")

df_filtered = pd.DataFrame(data['sales'])
print(f"Found {len(df_filtered)} high-value Widget sales in North/South regions")
df_filtered

## 5. Work with JSON and Arrays

In [ ]:
# Query customers with JSON metadata
data = run_query("""
    query {
        customers {
            name
            email
            metadata
            tags
            created_at
        }
    }
""")

customers_df = pd.DataFrame(data['customers'])

# Extract tier from JSON metadata
customers_df['tier'] = customers_df['metadata'].apply(lambda x: x['tier'])
customers_df['notifications'] = customers_df['metadata'].apply(
    lambda x: x['preferences'].get('notifications', False)
)

customers_df[['name', 'tier', 'notifications', 'tags']]

## 6. Add Computed Fields

In [ ]:
# Add computed field to calculate total value
@duckql.computed_field("sales", "total_value")
def total_value(obj) -> float:
    return round(obj['amount'] * obj['quantity'], 2)

# Query with computed field
data = run_query("""
    query {
        sales(limit: 5) {
            product
            amount
            quantity
            total_value
        }
    }
""")

pd.DataFrame(data['sales'])

## 7. Custom Analytics Queries

In [ ]:
# Add custom resolver for analytics
@duckql.resolver("salesAnalytics")
async def sales_analytics(root, info, start_date: str, end_date: str) -> dict:
    sql = """
        SELECT 
            product,
            region,
            COUNT(*) as transactions,
            SUM(amount) as total_amount,
            AVG(amount) as avg_amount,
            SUM(quantity) as total_quantity
        FROM sales
        WHERE date BETWEEN $1 AND $2
        GROUP BY product, region
        ORDER BY total_amount DESC
    """
    result = await duckql.executor.execute_query(
        sql, {"p0": start_date, "p1": end_date}
    )
    return {"analytics": result.rows}

# Use the custom resolver
data = run_query("""
    query {
        salesAnalytics(start_date: "2024-01-01", end_date: "2024-01-31") {
            analytics {
                product
                region
                transactions
                total_amount
                avg_amount
                total_quantity
            }
        }
    }
""")

analytics_df = pd.DataFrame(data['salesAnalytics']['analytics'])
analytics_df.head(10)

## 8. Visualization

In [ ]:
# Prepare data for visualization
pivot = analytics_df.pivot(index='product', columns='region', values='total_amount')

# Create visualization
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Bar chart by region
pivot.plot(kind='bar', ax=ax1)
ax1.set_title('Sales by Product and Region')
ax1.set_xlabel('Product')
ax1.set_ylabel('Total Sales')

# Pie chart of total sales by product
product_totals = analytics_df.groupby('product')['total_amount'].sum()
product_totals.plot(kind='pie', ax=ax2, autopct='%1.1f%%')
ax2.set_title('Total Sales Distribution by Product')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

## 9. Advanced: Parameterized Queries

In [ ]:
# Define a parameterized query
SALES_BY_DATE_RANGE = """
    query SalesByDateRange($startDate: String!, $endDate: String!, $minAmount: Float) {
        sales(
            where: {
                _and: [
                    { date: { gte: $startDate } }
                    { date: { lte: $endDate } }
                    { amount: { gte: $minAmount } }
                ]
            }
            orderBy: { date: ASC }
        ) {
            date
            product
            amount
        }
    }
"""

# Execute with variables
data = run_query(SALES_BY_DATE_RANGE, {
    "startDate": "2024-01-15",
    "endDate": "2024-01-20",
    "minAmount": 700
})

df_range = pd.DataFrame(data['sales'])
print(f"Found {len(df_range)} high-value sales in date range")
df_range.groupby('product')['amount'].agg(['count', 'sum', 'mean'])

## 10. Export Results

In [ ]:
# You can easily export results to various formats

# To CSV
analytics_df.to_csv('sales_analytics.csv', index=False)

# To Excel with multiple sheets
with pd.ExcelWriter('analysis_results.xlsx') as writer:
    analytics_df.to_excel(writer, sheet_name='Analytics', index=False)
    df_filtered.to_excel(writer, sheet_name='Filtered Sales', index=False)
    customers_df.to_excel(writer, sheet_name='Customers', index=False)

# Or back to DuckDB for further analysis
conn.execute("CREATE TABLE analytics_results AS SELECT * FROM analytics_df")

print("Results exported successfully!")